<a href="https://colab.research.google.com/github/admiralRus/Mayn/blob/main/DzadText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # загружаем библиотеки
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import pymorphy2
import dask.bag as db
import string

In [ ]:
#устанавливаем библиотеку
!pip install pymorphy2

In [ ]:
#скопируем текст

In [ ]:
#прочитаем содержимое файла, чтобы понять, что текст есть
with open('input.txt', 'r') as file:
  file_contents = file.readlines()
  db.read_text('input.txt').str.strip().str.split('\n')

In [ ]:
# Инициализация pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Функция для обработки текста и извлечения ключевых слов
def process_text(text):
    words = [word.lower().translate(str.maketrans('', '', string.punctuation)) for word in text]
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return [word for word in lemmatized_words if len(word) >= 3]

# Считываем текст из файла 'input.txt'
data = db.read_text('input.txt').str.strip().str.split()

# Создаем Dask Bag
bag = db.from_sequence(data)

# Применяем функцию к тексту
keywords = bag.map(process_text)

# Объединяем ключевые слова в один список
all_keywords = keywords.flatten()

# Подсчитываем количество вхождений каждого ключевого слова
keyword_counts = all_keywords.frequencies()

# Выводим результат
print(keyword_counts.compute())

In [ ]:
#выведем в датафрейм результаты, для анализа и группировки данных
result = pd.DataFrame(keyword_counts, columns = ['Слова', 'Количество'])
result

In [ ]:
#отсортируем слова по уменьшению количества повторений в тексте
#выведем 50 слов
result = result.sort_values(by="Количество", ascending=False).head(50)
result

In [ ]:
#построим диагрмамму плотности распределения слов
sns.kdeplot(result["Количество"])

In [ ]:
#удалим некотоыре слова, чтобы посмотиреть несколько популярных слов в тексте, которые имеют свое определение
result_fltr = result[~result["Слова"].str.contains("быть|они|что|как|это|свой|весь|тот|всё|ещё")]
result_fltr

В домашнем задании провели анализ рассказа Артура Конана Дойла "Как капитан Шарки и Стивен Крэддок перехитрили друг друга" из школьной программы внеклассного чтения. Самым пипоулярными словами в тексе оказались имена главных героев капитанов - Крэддок и Шарки.